# Agent Systems - Introduction to Cellular Automata (16 points)

# Cellular Automata

A Cellular Automata (abbreviated CA) is a **discrete, abstract computational system** ([Stanford Encyclopedia of Philospohy](https://plato.stanford.edu/entries/cellular-automata/)) used in various areas of science. Cellular Automata can simulate a wide range of processes in biology, chemistry etc. CA was invented by John von Neumann and Stanisław Ulam in the 1940s. A well-known specialist in CAs is Stephen Wolfram — creator of *Mathematica* and founder of *WolframAlpha*.

### CA consists of the following elements:

##### Grid
*n*-dimensional regular and discrete grid containing cells.
##### Cell
A discrete element of the grid. Each cell can be in one of a finite number of states. All states are updated simultaneously according to transition rules.
##### Transition rule
A mathematical function returning a new state (at time = *t*+1) based on to the current states of the cell (at time = *t*) and its neighbours (determined by the neighbourhood).  
##### Neighbourhood
The two most popular neighbourhoods (2D) are:
* Moore neighbourhood (8 cells):

![image](https://upload.wikimedia.org/wikipedia/commons/8/86/CA-Moore.svg)

<center>(source: <a href="https://commons.wikimedia.org/wiki/File:CA-Moore.svg">wikimedia.org</a>)</center>

* von Neumann neighbourhood (4 cells):

![image](https://upload.wikimedia.org/wikipedia/commons/2/26/CA-von-Neumann.svg)

<center>(source: <a href="https://commons.wikimedia.org/wiki/File:CA-von-Neumann.svg">wikimedia.org</a>)</center>

# Conway's Game of Life

Invented in 1970 by John Conway, **Game of Life** is the first and probably best-known example of Cellular Automata. The evolution of this automata is fully determined by its initial state (zero-player game).

The game centres around an infinite, two-dimensional grid of square cells. Each cell can be in one of the two  states: alive (*1*) or dead (*0*). In the original version each cell has 8 neighbours (Moore neighbourhood).

#### A new state is determined according to the following rules:

* Living cell remains **<span style="color:green">alive</span>** (doesn't change state) only if it has **<span style="color:blue">2</span>** or **<span style="color:blue">3</span>** living neighbours, otherwise it becomes **dead**.
* Dead cell surrounded by exactly **<span style="color:green">3</span>** living cells **<span style="color:green">becomes alive</span>**.

This set of rules os often denoted as **<span style="color:blue">23</span>/<span style="color:green">3</span>**.

All states are updated at the same time (simultaneously) resulting in a new **generation**. The initial state of the grid is sometimes called a **seed**. 



## Patterns

Many interesting patterns can be observed on the board. Common pattern belongs to one of the three following groups :
* **still life** (static) — structures that don't change between generations.
    + block ![image](https://upload.wikimedia.org/wikipedia/commons/9/96/Game_of_life_block_with_border.svg)
    + beehive ![image](https://upload.wikimedia.org/wikipedia/commons/6/67/Game_of_life_beehive.svg)
    + loaf ![image](https://upload.wikimedia.org/wikipedia/commons/f/f4/Game_of_life_loaf.svg)
* **oscillators** — structures that return to their initial states after a finite number of generations.
    + beacon ![image](https://upload.wikimedia.org/wikipedia/commons/1/1c/Game_of_life_beacon.gif)
    + blinker ![image](https://upload.wikimedia.org/wikipedia/commons/9/95/Game_of_life_blinker.gif)
    + pulsar ![image](https://upload.wikimedia.org/wikipedia/commons/0/07/Game_of_life_pulsar.gif)
* **spaceships** — structures that move across the grid.
    + **glider** — probably the most famous pattern ![image](https://upload.wikimedia.org/wikipedia/commons/f/f2/Game_of_life_animated_glider.gif)    
    + LWSS ![image](https://upload.wikimedia.org/wikipedia/commons/3/37/Game_of_life_animated_LWSS.gif)
    + MWSS ![image](https://upload.wikimedia.org/wikipedia/commons/4/4e/Animated_Mwss.gif)
    + HWSS ![image](https://upload.wikimedia.org/wikipedia/commons/4/4f/Animated_Hwss.gif)

<center>(source of animations: <a href="https://commons.wikimedia.org/">wikimedia.org</a>)</center>


Before moving to implementation, check the patterns in the [original version](https://playgameoflife.com) of the game. 

You can find complex patterns in the [lexcion](https://playgameoflife.com/lexicon).

# Python implementation of Conway's Game of Life

During the classes you'll be using Python and `pygame` framework. If you haven't heard about `pygame` before, you can look at a short introductory tutorial to this [framework](https://realpython.com/pygame-a-primer/).

## Basic implementation (3 points)
Below you will find a simple implementation of Game of Life in Python. Please fill in the missing parts: rules and grid generation.
* (1 point) Fill in the gaps in `initGrid` function. It should return a numpy array consisting of zeros and ones. Initialize the grid with random values.
* (2 points) Write the `update` function. Implement Conway's rules.

Run your code and observe CA's behaviour (a new window will appear, you can move to the next state using the right arrow key). Check some popular patterns. 



In [ ]:
import numpy as np
import itertools
import pygame

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
    K_RETURN
)

In [2]:
class Rule:
    def __init__(self, rule_string):
        survival, birth = rule_string.split('/')
        self.survival = set(int(x) for x in survival)
        self.birth = set(int(x) for x in birth)

    def should_live(self, cell_state, num_neighbors):
        if cell_state == 1:
            return num_neighbors in self.survival
        else:
            return num_neighbors in self.birth


In [5]:
class Simulation:
    def __init__(self, rows, cols, rule: Rule):
        self.rows = rows
        self.cols = cols
        self.grid = self.initGrid(rows, cols)
        self.w_width = 800
        self.w_height = 800
        pygame.init()
        self.screen = pygame.display.set_mode([self.w_width, self.w_height])
        self.rule = rule

    def run(self):
        running = True
        self.drawGrid(self.screen)
        while running:
            for event in pygame.event.get():   
                if event.type == QUIT:
                    running = False
                    pygame.quit()

                if event.type == KEYDOWN:
                    if event.key == K_RIGHT or event.key == K_UP or event.key == K_DOWN or event.key == K_LEFT:
                        self.grid = self.update()
                        self.drawGrid(self.screen)


    def update(self):
        newGrid = self.grid.copy()
        for i in range(self.rows):
            for j in range(self.cols):
                # Count live neighbors
                neighbors = self.grid[max(0, i-1):min(i+2, self.rows), max(0, j-1):min(j+2, self.cols)] #3x3 grid
                live_neighbors = np.sum(neighbors) - self.grid[i, j]
                if (self.rule.should_live(self.grid[i, j], live_neighbors)):
                    newGrid[i, j] = 1
                else:
                    newGrid[i, j] = 0
        return newGrid
        
    def initGrid(self, rows, cols):
        return np.random.randint(0,2,(rows,cols))

    def drawGrid(self, screen):
        alive_colour = (0,0,0)
        dead_colour = (255,255,255)
        rows, cols = self.grid.shape
        blockSize = (min(self.w_width, self.w_height)-max(rows, cols))/max(rows, cols)
        # For the sake of simplicity, we're skipping first and last rows & columns
        for x in range(1, rows-1):
            for y in range(1, cols-1):
                pos_x = (blockSize+1)*x
                pos_y = (blockSize+1)*y
                rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
                if self.grid[x][y] == 1:
                    pygame.draw.rect(screen, alive_colour, rect, 0)
                else:
                    pygame.draw.rect(screen, dead_colour, rect, 0)    
        pygame.display.flip()

In [ ]:
rows = 40
cols = 40
conway_rule = Rule("23/3")
simulation = Simulation(rows, cols, conway_rule)
simulation.run()

## Alternative Rules (3 points)

Create a **rule-independent implementation** (e.g. rules are passed as parameters). Please implement the following rules:
* Cities — 2345/45678.
* Coral — 45678/3.

Try to find other interesting rules. 


In [ ]:
cities_rule = Rule("2345/45678")
simulation = Simulation(rows, cols, cities_rule)
simulation.run()

In [ ]:
coral_rule = Rule("45678/3")
simulation = Simulation(rows, cols, coral_rule)
simulation.run()

## Periodic Boundaries (2 points)
In the implementation above, we're omitting cells located at the edges of the grid. Another way of dealing with border cells is to implement periodic boundaries. In order to implement them, you'll have to find a way of connectiong together the first and the last row, and the first and the last column. You can achieve this goal using e.g. modular arithmetic.

In [ ]:
class SimulationPeriodic(Simulation):
    def __init__(self, rows, cols, rule: Rule):
        super().__init__(rows, cols, rule)

    def update(self):
        newGrid = self.grid.copy()
        for i in range(self.rows):
            for j in range(self.cols):
                # Count live neighbors
                live_neighbors = 0
                for di in [-1, 0, 1]:
                    for dj in [-1, 0, 1]:
                        if di == 0 and dj == 0:
                            continue
                        ni = (i + di) % self.rows
                        nj = (j + dj) % self.cols
                        live_neighbors += self.grid[ni, nj]
                
                if (self.rule.should_live(self.grid[i, j], live_neighbors)):
                    newGrid[i, j] = 1
                else:
                    newGrid[i, j] = 0
        return newGrid
    
    def initGrid(self, rows, cols):
        # blinker on the edge
        grid = np.zeros((rows, cols))
        grid[0, 1] = 1
        grid[1, 1] = 1
        grid[-1, 1] = 1
        return grid
    
    def drawGrid(self, screen):
        alive_colour = (0,0,0)
        dead_colour = (255,255,255)
        rows, cols = self.grid.shape
        blockSize = (min(self.w_width, self.w_height)-max(rows, cols))/max(rows, cols)
        for x in range(rows):
            for y in range(cols):
                pos_x = (blockSize+1)*x
                pos_y = (blockSize+1)*y
                rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
                if self.grid[x][y] == 1:
                    pygame.draw.rect(screen, alive_colour, rect, 0)
                else:
                    pygame.draw.rect(screen, dead_colour, rect, 0)    
        pygame.display.flip()
    

simulation = SimulationPeriodic(rows, cols, conway_rule)
simulation.run()


## Additional modifications (4 points)
Implement additional features:
* (1 point) Counter displaying current time/step number ($t_0$ = *initial state*).
* (2 points) Previous states: Implement the possibility to go back to the previous states by using the left arrow key.
* (1 point) Add a field which will allow user to write the rule in a text field inside the GUI

In [ ]:
class ExtendedSimulation(Simulation):
    def __init__(self, rows, cols, rule: Rule):
        super().__init__(rows, cols, rule)
        self.history = []
        self.text = ''
        self.input_box = pygame.Rect(10, 50, 140, 32)
        self.color_inactive = pygame.Color('lightskyblue3')
        self.color_active = pygame.Color('dodgerblue2')
        self.foreground_color = pygame.Color('white')
        self.color = self.color_inactive
        self.active = False

    def update(self):
        newGrid = self.grid.copy()
        self.history.append(newGrid)
        return super().update()
        
    def go_back(self):
        if len(self.history) > 1:
            self.grid = self.history.pop()
            return self.grid
        return self.grid
    
    def run(self):
        running = True
        while running:
            running = self.handle_events()
            self.drawGrid(self.screen)
            pygame.time.delay(100)

    def drawGrid(self, screen):
        alive_colour = (0,0,0)
        dead_colour = (255,255,255)
        rows, cols = self.grid.shape
        
        padding = 100
        blockSize = (min(self.w_width, self.w_height)-max(rows, cols))/max(rows, cols)
        
        # Clear the screen
        screen.fill((0, 0, 0))  # Fill with black color
        
        for x in range(rows):
            for y in range(cols):
                pos_x = (blockSize+1)*x
                pos_y = (blockSize+1)*y + padding
                rect = pygame.Rect(pos_x, pos_y, blockSize, blockSize)
                if self.grid[x][y] == 1:
                    pygame.draw.rect(screen, alive_colour, rect, 0)
                else:
                    pygame.draw.rect(screen, dead_colour, rect, 0)    
        
        # Add counter displaying current step
        font = pygame.font.Font(None, 36)
        step_text = font.render(f"Step: {len(self.history)}", True, self.foreground_color)
        title_text = font.render("Game of Life", True, self.foreground_color)

        screen.blit(title_text, (self.w_width // 2 - title_text.get_width() // 2, 10))
        screen.blit(step_text, (10, 10))
        
        txt_surface = font.render(self.text, True, self.foreground_color)
        width = max(200, txt_surface.get_width()+10)
        self.input_box.w = width
        screen.blit(txt_surface, (self.input_box.x+5, self.input_box.y+5))
        pygame.draw.rect(screen, self.color, self.input_box, 2)
        pygame.display.flip()
        
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == QUIT:
                return False
            if event.type == KEYDOWN:
                if event.key == K_LEFT:
                    self.grid = self.go_back()
                elif event.key == K_RIGHT:
                    self.grid = self.update()
                elif event.key == K_RETURN:
                    try:
                        new_rule = Rule(self.text)
                        self.rule = new_rule
                        self.text = ''
                    except:
                        print("Invalid rule format")
                elif self.active:
                    if event.key == pygame.K_BACKSPACE:
                        self.text = self.text[:-1]
                    else:
                        self.text += event.unicode
            if event.type == pygame.MOUSEBUTTONDOWN:
                if self.input_box.collidepoint(event.pos):
                    self.active = not self.active
                else:
                    self.active = False
                self.color = self.color_active if self.active else self.color_inactive
        return True

simulation = ExtendedSimulation(rows, cols, conway_rule)
simulation.run()


# Rain (4 points)

Create a CA simulation of the rain:
* Change the neighbourhood definition. In this simulation, each cell will have only one neighbour i.e. the cell directly above it. 
* Create a function then randomly (with low probability e.g. 5%) changes the values in the first row of the grid to 6. Call this function in each `update` call. 
* Implement the following transition rules: 
    * If the current cell state is higher than 0, decrease the state by 1,
    * If the current cell state is 0 and the value at the cell directly above is greater than 0, change the next state to 6.
* Modifiy `drawGrid` function. Use different colours for different values from range 0-6.




In [ ]:
import random

class RainSimulation(Simulation):
    def update(self):
        newGrid = self.grid.copy()
        for i in range(self.rows):
            for j in range(self.cols):
                    current_cell = self.grid[i][j]
                    if (i>0):
                        above_cell = self.grid[i-1][j]
                        if current_cell > 0:
                            newGrid[i][j] = current_cell - 1
                        elif current_cell == 0 and above_cell > 0:
                            newGrid[i][j] = 6
                    elif (i == 0):
                        if (current_cell > 0):
                            newGrid[i][j] = current_cell - 1
                        elif (np.random.random() < 0.05):
                            newGrid[i][j] = 6
        return newGrid
    
    def initGrid(self, rows, cols):
        grid = np.zeros((rows, cols))
        for i in range(cols):
            if (np.random.random() < 0.05):
                grid[0][i] = 6
        return grid
    
    def drawGrid(self, screen):
        color_ranges_dict = {
            0: (0, 0, 0),
            1: (51, 51, 255),
            2: (102, 102, 255),
            3: (153, 153, 255),
            4: (204, 204, 255),
            5: (230, 230, 255),
            6: (255, 255, 255)
        }
        blockSize = (min(self.w_width, self.w_height)-max(rows, cols))/max(rows, cols)
        for i in range(self.rows):
            for j in range(self.cols):
                color = color_ranges_dict[self.grid[i][j]]
                pygame.draw.rect(screen, color, (j * blockSize, i * blockSize, blockSize, blockSize))
        pygame.display.flip()

rain_simulation = RainSimulation(rows, cols, conway_rule)
rain_simulation.run()